In [1]:
from pysal.lib import weights
import geopandas as gpd
import pandas as pd
import WeightsUtilities as WU
import numpy as np
from scipy.spatial.distance import cdist
from arcpy.stats import GenerateSpatialWeightsMatrix
from arcpy.stats import SpatialAutocorrelation
import arcpy

c:\Users\Lenovo\.conda\envs\arcgispro-spa-env\lib\site-packages\pysal\explore\segregation\network\network.py:16: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  "You need pandana and urbanaccess to work with segregation's network module\n"
c:\Users\Lenovo\.conda\envs\arcgispro-spa-env\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


### 调用WeightsUtilities模块将Dataframe存储为swm     
遇到的主要问题：生成swm后无法正常用于空间自相关分析    
解决方法：尝试调用arcpy.stats.GenerateSpatialWeightsMatrix模块生成

In [2]:
shp_path="F:\大创数据\中间产出的数据\云南省和黄淮海平原已处理好的火点\黄淮海平原逐月火点\黄淮海平原已处理好的火点_8月.shp"
swm_path="D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试\黄淮海平原已处理好的火点_8月.swm"
gdf = gpd.read_file(shp_path)

# 提取所需的字段（ID, X, Y, Z）
df = gdf[['ID', 'X', 'Y', 'Z']].copy()
coordinates = gdf[['X', 'Y', 'Z']].values

master_field="ID"
spatial_ref_name="World_Mercator"
# 设置距离阈值L0
L0 = 55000

# 计算空间权重矩阵
    #计算要素间的距离
    # coordinates = df[['X', 'Y']].values
distances = cdist(coordinates, coordinates)

    # 对得到的距离进行判断并赋值
spatial_weights = np.where((distances<=L0)&(distances!=0), 1, 0)

# 将空间权重矩阵保存到DataFrame中
spatial_weights_df = pd.DataFrame(spatial_weights)


# DataFrame转list转换函数
def df_to_list(df):
    result = []
    for idx, row in df.iterrows():
        element = idx
        neighbors = []
        weights = []
        for col, val in row.iteritems():
            if val != 0:
                neighbors.append(col)
                weights.append(val)
        result.append((element, neighbors, weights))
    return result
# DataFrame转dict转换函数
def df_to_dict(df):
    result = {}
    for idx, row in df.iterrows():
        element = idx
        neighbors = []
        weights = []
        for col, val in row.iteritems():
            if val != 0:
                neighbors.append(col)
                weights.append(val)
        result[element] = (np.array(neighbors), np.array(weights))
    return result

spatial_weights_dict=df_to_dict(spatial_weights_df)

#调用SWMWriter将spatial_weights_list写入swm文件
#格式如下：swm1.swm.writeEntry(要素,[邻居要素1,2,3……], [邻居要素的权重1,2,3……])
swm1=WU.SWMWriter(swm_path,master_field,spatial_ref_name,len(df),rowStandard=True,
                  wType=1,distanceMethod="Euclidean",threshold=55000)
for i in spatial_weights_dict:
    try: 
        if spatial_weights_dict[i][0] is None:
            swm1.swm.writeEntry(i,[],[])
        else:
            swm1.swm.writeEntry(i,spatial_weights_dict[i][0],spatial_weights_dict[i][1])
    except:
        print("---------",i,"---------")

swm1.close()


In [5]:
w={}
swm2=WU.SWMReader("D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试\黄淮海平原已处理好的火点_8月.swm")
for i in range(swm2.numObs):
    info=swm2.swm.readEntry()
    w[info[0]]=[info[2],info[3]]
w

{0: [array([279, 412, 534, 634, 923]), array([0.2, 0.2, 0.2, 0.2, 0.2])],
 1: [array([   2,   17,   18,   48,   50,   62,   68,   69,   70,  105,  107,
          108,  109,  130,  131,  132,  137,  142,  155,  156,  165,  167,
          168,  174,  199,  200,  206,  207,  215,  226,  227,  242,  255,
          258,  275,  276,  285,  303,  332,  340,  408,  414,  428,  429,
          437,  447,  477,  479,  480,  481,  490,  492,  493,  510,  531,
          532,  535,  536,  546,  547,  555,  560,  561,  596,  597,  635,
          655,  656,  657,  683,  699,  702,  703,  713,  749,  751,  780,
          785,  786,  794,  805,  806,  807,  814,  836,  860,  872,  881,
          904,  912,  946,  947,  958,  998,  999, 1001, 1059, 1077, 1078,
         1082, 1083, 1104, 1109, 1110, 1111, 1115, 1136, 1165, 1166, 1167,
         1168, 1174, 1176, 1191, 1199, 1215, 1222, 1223, 1237, 1238, 1239,
         1240, 1249, 1274, 1292, 1296, 1308, 1309, 1310, 1329, 1364, 1365,
         1372, 1374, 13

In [3]:


arcpy.env.overwriteOutput = True
arcpy.env.workspace="D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试"
featureset=arcpy.FeatureSet("F:\大创数据\中间产出的数据\云南省和黄淮海平原已处理好的火点\黄淮海平原逐月火点\黄淮海平原已处理好的火点_8月.shp")
SpatialAutocorrelation(featureset,Input_Field="FRP",Generate_Report=True,Conceptualization_of_Spatial_Relationships="GET_SPATIAL_WEIGHTS_FROM_FILE",
                       Weights_Matrix_File="D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试\黄淮海平原已处理好的火点_8月.swm")

ExecuteError: ERROR 000908: 任何要素都没有相邻要素。
执行(SpatialAutocorrelation)失败。


### 调用arcpy.stats.GenerateSpatialWeightsMatrix模块将df转为swm
结果：成功

In [11]:
shp_path="F:\大创数据\中间产出的数据\云南省和黄淮海平原已处理好的火点\黄淮海平原逐月火点\黄淮海平原已处理好的火点_8月.shp"
swm_path="D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试\黄淮海平原已处理好的火点_8月.swm"
txt_path="D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试\黄淮海平原已处理好的火点_8月.txt"
gdf = gpd.read_file(shp_path)

# 提取所需的字段（ID, X, Y, Z）
df = gdf[['ID', 'X', 'Y', 'Z']].copy()
coordinates = gdf[['X', 'Y', 'Z']].values

master_field="ID"
spatial_ref_name="World_Mercator"
# 设置距离阈值L0
L0 = 55000

# 计算空间权重矩阵
distances = cdist(coordinates, coordinates)

# 对得到的距离进行判断并赋值
spatial_weights = np.where((distances<=L0)&(distances!=0), 1, 0)

# 将空间权重矩阵保存到DataFrame中
# spatial_weights_df = pd.DataFrame(spatial_weights)


# 生成一个包含权重信息的列表
weight_info = []
for i in range(len(spatial_weights)):
    for j in range(len(spatial_weights)):
        weight = spatial_weights[i, j]
        if weight != 0:
            weight_info.append(f"{gdf.iloc[i]['ID']} {gdf.iloc[j]['ID']} {weight}")

# 将权重信息写入txt文件
with open(txt_path, 'w',encoding="ascii") as f:
    f.write("ID\n")
    for info in weight_info:
        f.write(f"{info}\n")